<a href="https://colab.research.google.com/github/Jufersoto/Organize-music/blob/main/Extract_playlists_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalar Librerias

In [ ]:
!pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
from sklearn.neural_network import MLPClassifier
#from skfuzzy.cluster import cmeans

from sklearn import metrics
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

import sklearn
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


# Sacar los datos de Spotify

In [ ]:
SPOTIFY_CLIENT_ID = "632358ae06694582b4083ca4b161f74d" 
SPOTIFY_CLIENT_SECRET = "366ffad5ed834380ba6d56bc300882bb" 
SPOTIFY_REDIRECT_URI = 'https://www.google.com/'
SCOPE = "playlist-modify-public playlist-read-private playlist-modify-private user-library-read"
username = 'jufersoto'
#https://open.spotify.com/user/jufersoto?si=5689a19438b747f3

In [ ]:
sp = spotipy.Spotify(
    auth_manager=spotipy.SpotifyOAuth(
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_CLIENT_SECRET,
        redirect_uri=SPOTIFY_REDIRECT_URI,
        scope=SCOPE, open_browser=False),
        requests_timeout=20, retries=10)
print('Authorization Sucessful!')

Authorization Sucessful!


In [ ]:
playlists = sp.user_playlists(username)
print(playlists['items'])
# Toma los datos, pero la primera vez ayuda con la autorizacion de usuario

[{'collaborative': False, 'description': 'This is Paté de Fuá. The essential tracks, all in one playlist.', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0BJ8es'}, 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DZ06evO0BJ8es', 'id': '37i9dQZF1DZ06evO0BJ8es', 'images': [{'height': None, 'url': 'https://thisis-images.scdn.co/37i9dQZF1DZ06evO0BJ8es-large.jpg', 'width': None}], 'name': 'This Is Paté de Fuá', 'owner': {'display_name': 'Spotify', 'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'}, 'href': 'https://api.spotify.com/v1/users/spotify', 'id': 'spotify', 'type': 'user', 'uri': 'spotify:user:spotify'}, 'primary_color': None, 'public': False, 'snapshot_id': 'MjgwOTIyOTAsMDAwMDAwMDA5NjczMjQwYTZhNGRhZTA1YzYxYmU5NjgzNmVhMDQ5Nw==', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DZ06evO0BJ8es/tracks', 'total': 44}, 'type': 'playlist', 'uri': 'spotify:playlist:37i9dQZF1DZ06evO0BJ8es'}, {'collaborative': False,

## Sacar los datos de las playlists y las características de las canciones directamente de Spotify

In [ ]:
# Datos de las playlists
playlists = pd.DataFrame.from_dict(playlists)
items = list(playlists['items'])
items = pd.DataFrame(items)
nombres = []
index = []
#objetivos = ['Hablan rapido', 'Uh lala señor francés', 'Top cross', 'Pa las caderas y el corazón', 'Tangos y boleros', 'Cumbiones', 'Iluminación', 'Canciones en ingles que me depriman o me duerman, lo que pase primero', 'Popsito', 'TimeTronica', 'A Andrés Cepeda le gusta esto', 'De guitarras y voces', 'Cowboy', 'Deep in Rain', 'Sun is shinning', 'Guapachoso']
objetivos = ['Pa las caderas y el corazón', 'Tangos y boleros', 'Cumbiones', 'Iluminación', 'Canciones en ingles que me depriman o me duerman, lo que pase primero', 'Popsito', 'TimeTronica', 'A Andrés Cepeda le gusta esto', 'De guitarras y voces', 'Cowboy', 'Deep in Rain', 'Sun is shinning', 'Guapachoso']

for i in range(len(items)):
  if items['name'][i] in objetivos:
    if i == playlists.index[i]:
      nombres.append(items['name'][i])
      index.append(i)
      #print(str(i)+", "+items['name'][i])

playlists_n = playlists.drop([i for i in range(len(playlists)) if i not in index])
playlists_n['name'] = nombres
Plts_ID = items['id']

#print(playlists_n.columns)



In [ ]:
# Datos de las canciones
track_list = [[],[],[],[]]
for i in range(len(nombres)):
  tracks = sp.playlist_tracks(Plts_ID[index[i]], offset=0, fields='items.track', additional_types=['track'])
  tracks = tracks['items']
  for j in range(len(tracks)):
    track_list[0].append(tracks[j]['track']['id'])
    track_list[1].append(nombres[i])
    track_list[2].append(tracks[j]['track']['name'])
    track_list[3].append(tracks[j]['track']['artists'][0]['name'])

track_df = pd.DataFrame({'track_id': track_list[0], 'playlist_name': track_list[1], 'track_name': track_list[2], 'artist':track_list[3]})

features_list = []

for k in range(len(track_list[0])):
  features = sp.audio_features(track_list[0][k])
  features_list.append(features[0])
  


ERROR:spotipy.client:Max Retries reached


SpotifyException: ignored

In [ ]:
# Limpieza previa de las listas y mandarlo a un csv
filtered_list = []
Fila_e = []
iteracion = 0
for x in features_list:
  iteracion = iteracion + 1
  if x is not None:
    filtered_list.append(x)
  else:
    Fila_e.append(iteracion)

features_df = pd.DataFrame(filtered_list)
df_concat = pd.concat([track_df, features_df], axis=1)

Datos = df_concat.drop(['type', 'id', 'uri', 'track_href', 'analysis_url', 'time_signature'], axis=1)
Datos.to_csv('Mis_canciones.csv')
 

## Sacar las canciones que me gustan

In [ ]:
# Saber mis canciones
datos_totales = []
fav_list = []
offset_actual = 0
nomb_canciones = []
nomb_artistas = []
while len(datos_totales) < 5000:
  Favorites = sp.current_user_saved_tracks(limit=50, offset=offset_actual)
  
  Favorites = Favorites['items']
  Favorites = pd.DataFrame(Favorites)
  if len(Favorites)==0:
    break
  Favorites = Favorites['track'].values
  Fav_id = [d['id'] for d in Favorites]
  Fav_name = [d['name'] for d in Favorites]
  Fav_Art = [d['artists'][0]['name'] for d in Favorites]
  nuevos_datos = Fav_id
  
  datos_totales += Fav_id
  nomb_canciones += Fav_name
  nomb_artistas += Fav_Art
  offset_actual += len(nuevos_datos)
  for t in range(len(Fav_id)):
    features = sp.audio_features(Fav_id[t])
    fav_list.append(features[0])

Total_nomb = pd.DataFrame(nomb_canciones, columns=['track_name'])
Total_art = pd.DataFrame(nomb_artistas, columns=['artist'])
Total_id = pd.DataFrame(datos_totales, columns=['track_id'])
Todas = pd.DataFrame(fav_list)
#7AoROkMQpKuUsJuNzmBykU

In [ ]:

df_junto = pd.concat([Total_id, Total_nomb, Total_art, Todas], axis=1)

df_junto = df_junto.drop(['type', 'id', 'uri', 'track_href', 'analysis_url', 'time_signature'], axis=1)
df_junto.to_csv('Todas_mis_canciones.csv')


# Revisión mejor modelo



In [ ]:
#Canciones = Datos
Canciones = pd.read_csv('Mis_canciones.csv')
Canciones = Canciones.dropna()
nombres = Canciones['track_name']

# Selección de características y etiquetas
X = Canciones[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']]
y = Canciones['playlist_name']

# División de train y test y escalamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=46,test_size=0.3, stratify=y)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)




## Iteraciones de todos los modelos

In [ ]:
# Matriz de las iteraciones 
models = [[SVC(), RandomForestClassifier(), LogisticRegression(),KNeighborsClassifier(),MLPClassifier(max_iter=1000)],[],
          ["Máquina de Soporte Vectorial", "Random Forest", "Regresión Logística", "KNN", "Redes Neuronales"]]

models[1].append([{'kernel': ['rbf'], 'C': [0.1, 1, 10], 'gamma': [0.001, 0.01, 0.1]},
                  {'kernel': ['poly'], 'C': [0.1, 1, 10], 'degree': [2, 3], 'coef0':[0, 1]}])
models[1].append({'n_estimators': [10, 100, 1000], 'max_depth': [2, 5, 10],
                  'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]})
models[1].append({'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']})
models[1].append({'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance'], 'p': [1, 2, 3]})
models[1].append({'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)], 'activation': ['tanh', 'relu'], 
                  'solver': ['sgd', 'adam'], 'alpha': [0.01], 'learning_rate': ['adaptive']})

# Predicciones y los scores de cross-validación
y_pred = []
cv_scores = []
mejores_params = []
for cont in range(5):
  clf = GridSearchCV(models[0][cont], 
    param_grid=models[1][cont], 
    cv=5, 
    scoring='f1_macro')

  clf.fit(X_train, y_train)
  y_pred.append(clf.predict(X_test))

  cv_scores.append(cross_val_score(models[0][cont], X_train, y_train, cv=5, scoring='f1_macro'))
  mejores_params.append(clf.best_params_)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceba

In [ ]:
avg_scores = [np.mean(scores) for scores in cv_scores]

index1 = np.argmax(avg_scores)
best_Cross_model = models[2][index1]
best_Cross_score = avg_scores[index1]
best_p1 = mejores_params[index1]

print("Mejor modelo:", best_Cross_model)
print("Score de cross-validación del mejor modelo:", best_Cross_score)
print("Mejores parametros:", best_p1)
print("Reporte de puntajes:")
print(classification_report(y_test, y_pred[index1], zero_division=1))

# Implementando los mejores modelos

In [ ]:
if best_Cross_model == "Máquina de Soporte Vectorial":
  modelo = SVC(**best_p1)
elif best_Cross_model == "Random Forest":
  modelo = RandomForestClassifier(**best_p1)
elif best_Cross_model == "Regresión Logística":
  modelo = LogisticRegression(**best_p1)
elif best_Cross_model == "KNN":
  modelo = KNeighborsClassifier(**best_p1)
elif best_Cross_model == "Redes Neuronales":
  modelo = MLPClassifier(**best_p1,max_iter=100)

modelo.fit(X, y)
y_prob = modelo.predict_proba(X_test)

# Encontrar la afinidad

In [ ]:
umbral = 0.4

top_labels = np.argsort(y_prob, axis=1)[:, ::-1] 
top_prob = np.sort(y_prob, axis=1)[:, ::-1]

# Suma acumulativa por  casilla, el ultimo debe dar 1.
sum = np.cumsum(top_prob, axis=1) 

filt_labels=[]
filt_prob=[]
for i in range(top_labels.shape[0]):
  max_labels = 0
  row = sum[i]
  #Encuentra el ultimo valor en donde false cambia a true, y el +1 dice cual es ese primer valor de true
  num_labels = np.argmax(row >= umbral) + 1 

  if num_labels > max_labels:
    max_labels = num_labels
    
  # Ubica tanto el la etiqueta, como las probabilidades de esa etiqueta
  filt_labels.append(top_labels[i, :max_labels])
  filt_prob.append(top_prob[:, :max_labels])

top_labels = []
for row in filt_labels:
  row_labels = [objetivos[i] for i in row]
  top_labels.append(row_labels)

# Calcular las predicciones y el f1-score
y_pred = [labels[0] if labels else None for labels in top_labels]
f1 = f1_score(y_test, y_pred, average='micro')
print("F1-score:",f1)

In [ ]:
Mis_Canciones = pd.read_csv('Todas_mis_canciones.csv')
identificacion = Mis_Canciones[['track_id','track_name','artist']]
# Selección de características y etiquetas
Caracteristicas = Mis_Canciones[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']]
y_caract = modelo.predict_proba(Caracteristicas)


umbral = 0.4 # La suma de las playlists debe ser menor a 40%

top_labels = np.argsort(y_caract, axis=1)[:, ::-1]
top_prob = np.sort(y_caract, axis=1)[:, ::-1]

sum = np.cumsum(top_prob, axis=1)
filt_labels=[]
filt_prob=[]
for i in range(top_labels.shape[0]):
  max_labels = 0
  row = sum[i]
  num_labels = np.argmax(row >= umbral) + 1

  if num_labels > max_labels:
    max_labels = num_labels
  
  filt_labels.append(top_labels[i, :max_labels])
  filt_prob.append(top_prob[i, :max_labels])

top_labels = []
for row in filt_labels:
  row_labels = [objetivos[i] for i in row]
  top_labels.append(row_labels)

# Organiza las canciones por su afinidad hacia las playlists
df_labels = pd.DataFrame(top_labels)
df_labels.columns = ['Playlist1', 'Playlist2', 'Playlist3','Playlist4','Playlist5','Playlist6']
df_prob = pd.DataFrame(filt_prob)
df_prob.columns = ['probabilidad de Pl1', 'probabilidad de Pl2', 'probabilidad de Pl3','probabilidad de Pl4','probabilidad de Pl5','probabilidad de Pl6']
df_listas = pd.concat([identificacion,df_labels, df_prob], axis=1)

df_listas.to_csv('Canciones_organizadas.csv')
print("Lista del resultado de las canciones:")
print(df_listas)

